# Préparation des données avec ELMO à la place de Word2Vec

In [1]:
import pandas as pd
import numpy as np

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
dataset = pd.read_csv('data/train_cap2018.csv')

In [3]:
from sklearn.model_selection import train_test_split
test_size = 0.2
random_state = 42

In [4]:
train, test = train_test_split(dataset, test_size=test_size, random_state=random_state, shuffle=True, stratify=dataset.loc[:,'level1'])
print(train.shape)
print(test.shape)

(21848, 60)
(5462, 60)


In [5]:
X_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]
print(X_train.shape)
print(y_train.shape)

(21848, 59)
(21848,)


In [6]:
y_train = y_train.replace({"A1": 0, "A2" : 1, "B1" : 2, "B2" : 3, "C1" : 4, "C2" : 5})

In [7]:
#X_train_fulltext = X_train.loc[:, 'fulltext'].iloc[:10]
#y_train = y_train[:10]
#print(X_train_fulltext)

X_train_fulltext = X_train.loc[:, 'fulltext']

In [8]:
training_size = 0.7

X_train = X_train_fulltext[0:int(X_train_fulltext.shape[0]*training_size)]
y_train_full = y_train
y_train = y_train_full[0:int(X_train_fulltext.shape[0]*training_size)]

X_val = X_train_fulltext[int(X_train_fulltext.shape[0]*training_size):]
y_val = y_train_full[int(X_train_fulltext.shape[0]*training_size):]

In [9]:
y_train = np.array(y_train.reset_index().iloc[:,1])
y_val = np.array(y_val.reset_index().iloc[:,1])

In [10]:
print(X_val.shape)
print(y_val.shape)
print(X_train.shape)
print(y_train.shape)

(6555,)
(6555,)
(15293,)
(15293,)


# Programmation avec ELMO

Récupération d'un module ELMO déjà entrainé.

In [11]:
import tensorflow as tf
import tensorflow_hub as hub

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [12]:
def get_next_batch(X, iteration, batch_size):
    X_batch = X[iteration*batch_size:(iteration+1)*batch_size]
    return X_batch

In [81]:
def create_graph_RNN_and_train(X_train, y_train, X_val, y_val, n_steps, n_neurons=500, activation=tf.nn.relu, 
                               dropout_in=0, dropout_out=0, class_weights=[1, 1, 1, 1, 1, 1], learning_rate=0.001, 
                               n_epochs=100, batch_size=200, max_checks_without_progress=3):
    
    tf.reset_default_graph()
    
    X = tf.placeholder(dtype=tf.string, shape=[None], name="X")
    y = tf.placeholder(tf.int64, shape=[None], name="y")
    
    elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=False)
    X_elmo = elmo(X, signature="default", as_dict=True)["elmo"]
    #X_elmo = tf.cast(X_elmo, tf.float16)
    sequence_length = tf.shape(X_elmo)[1]*tf.ones(tf.shape(X_elmo)[0], dtype=tf.int32)
    
    dropout_in_placeholder = tf.placeholder_with_default(tf.constant(0.0, dtype=tf.float32), ())
    dropout_out_placeholder = tf.placeholder_with_default(tf.constant(0.0, dtype=tf.float32), ())
    
    basic_cell = tf.contrib.rnn.GRUCell(num_units=n_neurons, activation=activation)
    basic_cell = tf.contrib.rnn.DropoutWrapper(basic_cell, input_keep_prob=1-dropout_in_placeholder, output_keep_prob=1-dropout_out_placeholder, dtype=tf.float32)
    outputs, states = tf.nn.dynamic_rnn(basic_cell, X_elmo, sequence_length=sequence_length, dtype=tf.float32)

    logits = tf.layers.dense(inputs=states, units=n_outputs, name="logits")
    inference = tf.nn.softmax(logits, name="inference")

    with tf.name_scope("loss"):
        #loss = cost(inference, y)

        class_weights_tf = tf.constant(class_weights)
        weights = tf.gather(class_weights_tf, y)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=logits, weights=weights)
        #xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y) #ancienne version (sans poids)
        #loss = tf.reduce_mean(xentropy, name="loss")

    with tf.name_scope("train"):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss)

    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(tf.cast(logits, tf.float32), y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    summary_writer = tf.summary.FileWriter("./summary", tf.get_default_graph())

    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
    n_batches_per_epoch = X_train.shape[0] // batch_size
    print("Nombre de batchs par epoch =", n_batches_per_epoch)
    
    best_loss = np.infty
    checks_without_progress = 0
    early_stopping = False
    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epochs):
            for iteration in range(n_batches_per_epoch):
                X_batch = get_next_batch(X_train, iteration, batch_size)
                y_batch = y_train[iteration*batch_size:(iteration+1)*batch_size]

                sess.run(training_op, feed_dict={X: X_batch, y: y_batch, dropout_in_placeholder: dropout_in, dropout_out_placeholder: dropout_out})
                #print(sess.run(X_elmo, feed_dict={X: X_batch, y: y_batch, dropout_in_placeholder: dropout_in, dropout_out_placeholder: dropout_out}))
                if (iteration+1)%10==0:
                    print("Batch n°", iteration+1)
                if (iteration+1)%100==0:
                    # attention l'évaluation du coût est faite en faisant une moyenne de moyennes,
                    # il faut donc que nb_examples_to_evaluate soit un multiple de batch_size_loss_eval
                    nb_examples_to_evaluate = np.amin([1000, X_train.shape[0], X_val.shape[0]])
                    batch_size_loss_eval = 25

                    i=0
                    k=0
                    loss_train=0
                    while i < nb_examples_to_evaluate:
                        X_batch = get_next_batch(X_train, k, batch_size_loss_eval)
                        y_batch = get_next_batch(y_train, k, batch_size_loss_eval)
                        temp = loss.eval(feed_dict={X: X_batch, y: y_batch})
                        #print(temp)
                        loss_train = loss_train + temp
                        i = i + batch_size_loss_eval
                        k = k + 1
                    #print(loss_train)
                    #print(k)
                    loss_train = loss_train / k
                    print(epoch, "Loss training on", nb_examples_to_evaluate, "examples:", loss_train)

                    i=0
                    k=0
                    loss_val=0
                    while i < nb_examples_to_evaluate:
                        X_batch = get_next_batch(X_val, k, batch_size_loss_eval)
                        y_batch = get_next_batch(y_val, k, batch_size_loss_eval)
                        temp = loss.eval(feed_dict={X: X_batch, y: y_batch})
                        loss_val = loss_val + temp
                        i = i + batch_size_loss_eval
                        k = k + 1
                    loss_val = loss_val / k
                    print(epoch, "Loss validation on", nb_examples_to_evaluate, "examples:", loss_val)

                    if loss_val < best_loss:
                        save_path = saver.save(sess, "./natural_language_classifier.ckpt")
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                        if checks_without_progress >= MAX_CHECKS_WITHOUT_PROGRESS:
                            print("Early stopping!")
                            early_stopping = True
                            break
                if early_stopping:
                    break
            
    return inference, X

In [82]:
n_steps = 450 #taille maximale des textes (rendue fixe)
print("Nombre maximal de mots par texte (fixe) =", n_steps)
n_neurons = 5000
activation = tf.nn.relu
n_outputs = 6
class_weights = [1, 1, 1, 1, 1, 1] #poids de la fonction de coût
learning_rate = 0.001

n_epochs = 100
batch_size = 30

MAX_CHECKS_WITHOUT_PROGRESS = 3

Nombre maximal de mots par texte (fixe) = 450


In [83]:
inference, X = create_graph_RNN_and_train(X_train, y_train, X_val, y_val, n_steps, n_neurons=n_neurons, 
                                          activation=activation, class_weights=class_weights,
                                           learning_rate=learning_rate, n_epochs=n_epochs, 
                                          batch_size=batch_size, max_checks_without_progress=MAX_CHECKS_WITHOUT_PROGRESS)

#0.6 sur les ensembles d'entrainement et d'évaluation avec 500 neurones

INFO:tensorflow:Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with aggregation/scaling
INFO:tensorflow:Initialize variable module/aggregation/weights:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with aggregation/weights
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_0:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with bilm/CNN/W_cnn_0
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_1:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with bilm/CNN/W_cnn_1
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_2:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with bilm/CNN/W_cnn_2
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_3:0 from checkp

Nombre de batchs par epoch = 509
Batch n° 10
Batch n° 20
Batch n° 30
Batch n° 40
Batch n° 50
Batch n° 60
Batch n° 70
Batch n° 80
Batch n° 90
Batch n° 100
0 Loss training on 1000 examples: 1.0612540900707246
0 Loss validation on 1000 examples: 0.9590685188770294
Batch n° 110
Batch n° 120
Batch n° 130
Batch n° 140
Batch n° 150
Batch n° 160
Batch n° 170
Batch n° 180
Batch n° 190
Batch n° 200
0 Loss training on 1000 examples: 0.5899741400033236
0 Loss validation on 1000 examples: 0.5774521581828594
Batch n° 210
Batch n° 220
Batch n° 230
Batch n° 240
Batch n° 250
Batch n° 260
Batch n° 270
Batch n° 280
Batch n° 290
Batch n° 300
0 Loss training on 1000 examples: 0.5725694600492716
0 Loss validation on 1000 examples: 0.5708290565758943
Batch n° 310
Batch n° 320
Batch n° 330
Batch n° 340
Batch n° 350
Batch n° 360
Batch n° 370
Batch n° 380
Batch n° 390
Batch n° 400
0 Loss training on 1000 examples: 0.41005709022283554
0 Loss validation on 1000 examples: 0.4349073935300112
Batch n° 410
Batch n° 4

In [84]:
costs = np.array([[0,1,2,3,4,6],[1,0,1,4,5,8],[3,2,0,3,5,8],[10,7,5,0,2,7],[20,16,12,4,0,8],[44,38,32,19,13,0]])
names = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']

In [85]:
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes=['A1', 'A2', 'B1', 'B2', 'C1', 'C2'],
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [86]:
from sklearn.metrics import confusion_matrix
print_confusion = True
def cost(y_pred, y_true, normalize=True):
    confusion = confusion_matrix(y_true, y_pred)
    res = (1/y_true.shape[0]) * np.sum(np.multiply(costs, confusion))
    
    if print_confusion:
        # Compute confusion matrix
        cnf_matrix = confusion_matrix(y_true, y_pred)
        np.set_printoptions(precision=2)

        # Plot normalized confusion matrix
        plt.figure()
        plot_confusion_matrix(cnf_matrix, normalize=normalize, title='Normalized confusion matrix')

        plt.show()
    return res

In [ ]:
print_confusion = True
batch_size = 50

saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "./natural_language_classifier.ckpt")
    i=0
    iteration=0
    y_pred = []
    while i < X_val.shape[0]:
        X_batch = get_next_batch(X_val, iteration, batch_size)
        print(X_batch)
        temp = sess.run(inference, feed_dict={X: X_batch})
        print(temp)
        y_pred.extend(np.argmax(temp, axis=1))
        print(y_pred)
        i = i + batch_size
        iteration = iteration + 1
print(cost(y_pred, y_val))

INFO:tensorflow:Restoring parameters from ./natural_language_classifier.ckpt
17858    \n\n      Hi! my name is DanieleI'm sixty-five...
6389     \n\n      In France, when you're invited for a...
5119     \n\n      Home medicines : the best medicines ...
4710     \n\n      I think the meat was a little bit un...
23696    \n\n      Hi, My name is Fatiha. I'm 24 years ...
13075    \n\n      Hi Mom! On Saturday, 23th September,...
15900    \n\n      From: Mary Hey you, I would like to ...
22765    \n\n      I always go to the big shops. There ...
23734    \n\n      Dear Ms Thomas In the offcice ,we ar...
1375     \n\n      Hi, Anna! I've found an absolutely a...
4230     \n\n      sorry, but the text is not in line w...
21252    \n\n      I usually shop on line. It easier fo...
4106     \n\n      During my life, I'd like win enough ...
8076     \n\n      From : Sabine Leygoute Teilliet To :...
11357    \n\n      john discovers that Isabella doesn't...
9820     \n\n      I worked for a litt

[[3.41618084e-03 9.95578587e-01 8.26136558e-04 9.99357799e-05
  5.12143743e-05 2.79882715e-05]
 [1.25023216e-05 7.37055507e-06 2.22209128e-04 9.99140501e-01
  3.69899557e-04 2.47380172e-04]
 [1.45313839e-04 1.34541653e-03 1.22476164e-02 9.75265563e-01
  4.35054023e-03 6.64565060e-03]
 [3.26955542e-02 1.07422747e-01 6.23192370e-01 2.90288944e-02
  1.61932781e-01 4.57276702e-02]
 [1.98449816e-05 9.99498606e-01 3.98896431e-04 6.37026533e-05
  6.05814694e-06 1.28059328e-05]
 [1.17004242e-06 1.29733780e-06 2.08783422e-06 9.99992013e-01
  4.55747340e-07 2.89863488e-06]
 [2.03405158e-03 9.93669808e-01 4.18526726e-03 8.42711815e-05
  2.37224031e-05 2.84303542e-06]
 [1.04299746e-04 9.99783456e-01 9.85060688e-05 8.87558599e-06
  1.87794865e-06 2.98488089e-06]
 [8.79817890e-05 4.93566549e-05 1.40408047e-05 9.99798954e-01
  4.82331507e-06 4.48245628e-05]
 [9.98652816e-01 8.32741323e-04 3.01200809e-04 1.33973299e-04
  7.65727164e-05 2.69339876e-06]
 [9.99379396e-01 3.82673636e-04 1.37587354e-04 5.2

[[6.58155244e-04 1.93746397e-04 7.72993743e-01 1.80773273e-01
  1.74521189e-02 2.79289614e-02]
 [1.17693515e-03 2.28424044e-03 6.90034181e-02 9.09861982e-01
  1.20924320e-02 5.58105763e-03]
 [9.97700751e-01 1.33445160e-03 4.30023938e-04 3.60549602e-04
  1.67398553e-04 6.85673240e-06]
 [1.44813776e-01 8.48017454e-01 6.04963023e-03 6.76119991e-04
  4.29783395e-04 1.33235435e-05]
 [1.04359619e-03 6.89561857e-05 1.87925100e-01 4.66321595e-03
  7.98934579e-01 7.36453198e-03]
 [9.63501952e-05 9.99828577e-01 6.51994633e-05 7.86444707e-06
  6.30728209e-07 1.42864815e-06]
 [9.98059213e-01 1.22315669e-03 4.59586183e-04 1.71275795e-04
  8.17130931e-05 5.03670208e-06]
 [1.32076809e-06 9.99992490e-01 4.76289051e-06 1.28149179e-06
  3.40664812e-08 7.02309038e-08]
 [9.98164356e-01 1.21195370e-03 3.81603400e-04 1.57006027e-04
  8.03470684e-05 4.89569447e-06]
 [1.32574969e-05 3.36254247e-09 9.99986768e-01 3.87490884e-09
  1.53698432e-09 2.90740505e-11]
 [9.68622088e-01 2.71083005e-02 1.76643359e-03 2.2

[[9.87048209e-01 1.14563052e-02 9.13035241e-04 3.81925289e-04
  1.80876275e-04 1.97870067e-05]
 [9.99484420e-01 3.69112793e-04 1.06643129e-04 2.58013370e-05
  1.33773010e-05 5.17003798e-07]
 [9.94525969e-01 3.86758894e-03 8.61866632e-04 5.99171850e-04
  1.32661953e-04 1.27302701e-05]
 [8.99194711e-05 9.77238536e-01 3.09943745e-04 1.90270022e-02
  1.50678959e-03 1.82780100e-03]
 [9.98232186e-01 1.00090937e-03 4.91563114e-04 1.83972617e-04
  8.59425054e-05 5.42199314e-06]
 [9.84772325e-01 7.84193538e-03 2.46906327e-03 3.37425759e-03
  1.45165424e-03 9.07598878e-05]
 [9.99442756e-01 4.28206666e-04 9.64927458e-05 2.12836021e-05
  1.04482042e-05 7.92524986e-07]
 [4.11486835e-05 9.99236822e-01 7.08634208e-04 1.12421840e-05
  1.12224575e-06 1.17271770e-06]
 [8.16656538e-08 2.97808334e-09 9.99998808e-01 9.69595135e-07
  5.56793509e-08 6.97084257e-09]
 [1.40133038e-01 8.55208457e-01 2.75859749e-03 1.53338886e-03
  2.61892972e-04 1.04688726e-04]
 [2.45937912e-07 4.51845361e-09 9.99999642e-01 3.1

[[1.28854372e-04 9.99766767e-01 8.99720908e-05 1.24408543e-05
  6.38667302e-07 1.39222107e-06]
 [3.79904890e-07 9.99995589e-01 3.20056802e-06 7.26289272e-07
  2.82162933e-08 4.25607816e-08]
 [9.98262823e-01 1.15601346e-03 3.19332554e-04 1.74105080e-04
  8.43613816e-05 3.43032889e-06]
 [6.59792244e-01 3.20145994e-01 6.60281302e-03 3.01287742e-03
  9.59954411e-03 8.46582581e-04]
 [1.04949072e-06 9.99988317e-01 1.01520354e-05 2.83363988e-07
  1.29204354e-08 1.22027672e-07]
 [9.97724116e-01 1.39908970e-03 5.36075328e-04 2.66395626e-04
  6.92702015e-05 5.08642006e-06]
 [7.67141290e-04 8.12651590e-04 3.49724978e-01 5.16186178e-01
  1.08617261e-01 2.38917526e-02]
 [2.23194491e-02 4.30270750e-03 5.16497254e-01 4.42295313e-01
  6.51767757e-03 8.06756504e-03]
 [9.87758219e-01 4.01545694e-04 3.08349472e-03 1.06024602e-03
  7.63580669e-03 6.07236325e-05]
 [9.91441131e-01 7.20294006e-03 5.61568537e-04 4.42518212e-04
  3.25230445e-04 2.66625957e-05]
 [6.33169059e-03 9.93082404e-01 5.01001487e-04 5.5

...

[[3.96916488e-14 1.00000000e+00 5.72974446e-10 1.50364594e-14
  2.68681630e-16 1.71443068e-16]
 [2.28479421e-05 9.99729097e-01 2.47274547e-05 1.36088580e-04
  2.45650826e-05 6.26155233e-05]
 [6.51279333e-06 9.99976039e-01 1.65245856e-05 5.49601054e-07
  2.40591135e-07 1.31709172e-07]
 [2.70180812e-04 1.34770322e-04 1.88033376e-02 8.47382188e-01
  1.30652070e-01 2.75747804e-03]
 [9.98624444e-01 9.56436445e-04 2.61328270e-04 1.00167075e-04
  5.45731345e-05 3.01640648e-06]
 [6.86064993e-07 2.38605679e-07 9.99996781e-01 2.02912724e-06
  2.05540744e-07 8.32290397e-08]
 [2.77429353e-04 3.62959472e-05 9.99302149e-01 2.57045525e-04
  1.04534469e-04 2.24772102e-05]
 [9.98378992e-01 9.02070722e-04 4.10828448e-04 2.35370157e-04
  6.90163361e-05 3.70304724e-06]
 [1.77600309e-01 8.19032192e-01 2.81543913e-03 4.20625292e-04
  9.66524676e-05 3.47183850e-05]
 [2.87476752e-04 9.99035120e-01 2.22109200e-04 2.91117933e-04
  6.28689741e-05 1.01345991e-04]
 [9.95819688e-01 2.16677296e-03 1.34183513e-03 4.7

[[5.65462415e-06 4.75655327e-07 9.99927998e-01 6.11467549e-05
  4.21237110e-06 5.96343227e-07]
 [9.97927666e-01 1.61453662e-03 2.94597878e-04 1.06794723e-04
  5.24410461e-05 3.89613433e-06]
 [9.43548977e-01 5.25609031e-02 1.58468867e-03 1.48367195e-03
  7.09711167e-04 1.12112044e-04]
 [9.61926673e-03 3.81762870e-02 9.45097089e-01 4.55031544e-03
  1.95370871e-03 6.03309134e-04]
 [9.92362142e-01 4.92895162e-03 1.38641323e-03 9.11213807e-04
  3.77979304e-04 3.33329117e-05]
 [2.51527550e-03 9.95162427e-01 1.79278245e-03 2.35930900e-04
  8.94108598e-05 2.04186625e-04]
 [9.97986913e-01 8.33595113e-04 7.41711177e-04 3.12909280e-04
  1.19973316e-04 4.80500466e-06]
 [1.70607748e-03 3.82871658e-05 9.97728169e-01 2.81702582e-04
  2.17765119e-04 2.80459444e-05]
 [9.97188866e-01 2.13860814e-03 3.93744704e-04 1.80930219e-04
  9.13704498e-05 6.45281352e-06]
 [1.04047591e-03 7.68966079e-01 2.27044865e-01 2.17558304e-03
  4.91089653e-04 2.81936460e-04]
 [7.03781916e-05 9.49164331e-01 4.94958125e-02 9.3

[[6.28483074e-04 3.69345595e-04 9.97262359e-01 1.29208702e-03
  3.39955790e-04 1.07766122e-04]
 [5.55627630e-04 9.99095917e-01 2.87839328e-04 4.11606597e-05
  1.05428517e-05 8.94985442e-06]
 [9.99440253e-01 4.64983663e-04 8.35649043e-05 6.69580550e-06
  4.34425601e-06 9.77678596e-08]
 [9.94711757e-01 4.18820186e-03 5.07849210e-04 3.40920291e-04
  2.33368453e-04 1.79203289e-05]
 [9.98447776e-01 1.07687805e-03 3.07746843e-04 1.16463605e-04
  4.77149479e-05 3.39578537e-06]
 [2.22498458e-03 9.92411673e-01 2.96864123e-03 2.15685088e-03
  9.45232387e-05 1.43242651e-04]
 [1.04869898e-07 3.71894633e-11 9.99999881e-01 8.47962145e-09
  8.40671888e-09 5.27757837e-11]
 [1.28787115e-01 8.09777498e-01 4.99546528e-02 6.59793150e-03
  3.71790095e-03 1.16492133e-03]
 [2.51531368e-04 1.73914115e-04 9.97194409e-01 2.05078186e-03
  2.80119595e-04 4.91944302e-05]
 [2.10793521e-02 9.76087272e-01 2.34603323e-03 3.03488050e-04
  1.33904527e-04 5.01128707e-05]
 [9.98550117e-01 1.00489869e-03 2.74994498e-04 1.4

[[7.16958166e-05 9.99363363e-01 5.22037852e-04 2.08542460e-05
  2.02426654e-05 1.76772528e-06]
 [1.45278580e-04 8.11201346e-07 9.99256670e-01 4.07833926e-04
  1.84588178e-04 4.80780682e-06]
 [4.56995840e-05 9.99949694e-01 3.01040222e-06 1.19120216e-06
  3.97676160e-07 9.23501702e-08]
 [9.94283259e-01 4.48019663e-03 6.98513992e-04 3.76008364e-04
  1.49352709e-04 1.25169472e-05]
 [9.98186290e-01 1.26343197e-03 3.36070545e-04 1.43697282e-04
  6.64277395e-05 4.06983281e-06]
 [1.90829619e-09 1.13828894e-11 1.00000000e+00 2.00408051e-10
  8.42974822e-12 9.18004821e-13]
 [2.77114188e-13 1.00000000e+00 3.90142807e-09 1.66968869e-13
  1.64724392e-15 1.42608203e-15]
 [9.96967256e-01 2.41278834e-03 3.72061302e-04 1.71819600e-04
  7.05596321e-05 5.48844628e-06]
 [2.19625463e-06 9.99979019e-01 1.85060799e-05 2.92306026e-07
  2.88873139e-08 1.64877729e-08]
 [4.31685112e-02 6.85289677e-04 9.40911710e-01 3.02119553e-03
  1.13205044e-02 8.92821001e-04]
 [1.78890795e-01 8.18704724e-01 1.22835103e-03 8.2

[[9.94644880e-01 4.08631004e-03 8.25706520e-04 2.90606695e-04
  1.40593504e-04 1.19305860e-05]
 [1.89865226e-04 9.99779403e-01 1.81260093e-05 6.74316061e-06
  3.43132911e-06 2.42592932e-06]
 [9.95826423e-01 3.26704676e-03 5.19427820e-04 2.56449450e-04
  1.20823977e-04 9.73622537e-06]
 [9.57094431e-01 4.00416553e-02 1.96775002e-03 6.12878939e-04
  2.31568119e-04 5.17612207e-05]
 [9.95799601e-01 3.26089840e-03 6.39216218e-04 1.82614531e-04
  1.09083638e-04 8.54498739e-06]
 [9.91472304e-01 7.04026222e-03 7.70736777e-04 5.28308272e-04
  1.70361353e-04 1.80726911e-05]
 [6.72771633e-02 8.17895830e-01 4.43548197e-03 6.89411582e-03
  8.23054984e-02 2.11918578e-02]
 [7.29909807e-04 9.98582840e-01 5.43319096e-04 8.03705625e-05
  3.14580211e-05 3.20707950e-05]
 [1.08052491e-05 9.99766052e-01 2.20691159e-04 1.82510814e-06
  2.09305242e-07 4.28192152e-07]
 [3.61960565e-06 2.85043058e-07 9.99992132e-01 2.86629438e-06
  9.33516560e-07 8.35602947e-08]
 [1.53197561e-06 9.99996066e-01 2.18388755e-06 2.4